## 📢 **Updates**
- **05 Jun 2021**: Table are logged into a run with bigger subset of datasets
- **31 May 2021**: Table are logged into an artifact with small subset of datasets

## 🎯 **Goal**
[Rainforest Connection Species Audio Detection competition](https://www.kaggle.com/c/rfcx-species-audio-detection/overview) contains thousands of audio clips with both species labels and their localized regions.  
Utilising the recently released feature (`wandb.Table`) by [Weight and Bias](https://wandb.ai/site), This notebook serves the followin purposes:
- Offers an off-the-shelf tabular tool to explore the datasets
- Walk you through how to create an awesome tool like this using W&B

## 📚 **References**
- [W&B Dataset and Predictions Viz Demo](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/dsviz/W%26B_Dataset_and_Predictions_Viz_Demo.ipynb)
- [Visualize Audio Data in W&B](https://wandb.ai/stacey/cshanty/reports/Visualize-Audio-Data-in-W-B--Vmlldzo1NDMxMDk)

Additionally, I would like to take this chance to thank [@ayuraj](https://www.kaggle.com/ayuraj) for introducing W&B to me and for all the amazing notebooks you have created!

## 🔍 **Try It Out Now!**
The tool is freely hosted in W&B and you can check it out here:  
[https://wandb.ai/alexlauwh/RFCX-EDA-v2?workspace=user-alexlauwh](https://wandb.ai/alexlauwh/RFCX-EDA-v2?workspace=user-alexlauwh)

The interface offers the following basic functions:
- Hear the localized audio for each species (include true positive, false positive and each songtype)  
- Inspect its associated melspectrogram
- Identify the time & frequency range from the melspectrogram associated to the species


Below is a snapshot of the interface:  
![](https://imgur.com/HOcgj0V.png)


In addition to the above functions, you can also:  
- filter sample by simple criteria (e.g. filter by `species_id = 12` as shown)  
![](https://imgur.com/f8D9cMP.png)

- aggregate samples for high-level analysis (e.g. group by `species_id` as shown)  
![](https://imgur.com/LngQqlz.png)

I Here I just highlighted key features that I think are useful. You could refer to [the documentation](https://docs.wandb.ai/guides/data-vis) for its full functionalities.

## ❓ **How To Create a Tabular Tool Like This**
Run the code below to reproduce the tabular tool using W&B, but before that there are a few steps you need to complete:
1. Apply a W&B account and get your W&B API key from **"User Settings"**  
![](https://i.imgur.com/PY0Ywuh.png)
2. Create an environment variable `wandb_api` in this kernel for the API key. To do this navigate to the panel below by **"Add-ons" >> "Secrets"**  
![](https://imgur.com/633GGXU.png)

The code will do the following:
1. Create a project and a run in your account
2. Create a `wandb.Table` object
3. Log the `wandb.Table` object into your run

In [ ]:
!pip install wandb --upgrade

In [ ]:
import tempfile
from pathlib import Path
from typing import Tuple

import wandb
from kaggle_secrets import UserSecretsClient

import librosa
import librosa.display

from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# WANDB CONFIG
# define your prefered project and run name
WANDB_PROJECT = 'RFCX-EDA-v2'
WANDB_RUN_NAME = 'RFCX-EDA-v2'
WANDB_JOB_TYPE = 'EDA'
# upload subset of data to artifact, instead of all data (avoid memory crash)
SAMPLE_N_PER_GROUP = 20

# DATA CONFIG
DATA_DIR = Path('/kaggle/input/rfcx-species-audio-detection')
TP_PATH = DATA_DIR/ 'train_tp.csv'
FP_PATH = DATA_DIR/ 'train_fp.csv'

# AUDIO/ PLOTTING CONFIG
SR = 48000
FMAX = int(SR/2)
FMIN = 0
LIBROSA_CONFIG = {
    'sr': SR,
    'n_fft': 2048,
    'hop_length': 512,
    'fmin': FMIN,
    'fmax': FMAX
}
DISPLAY_CONFIG = {
    'sr': SR, 
    'fmin': FMIN,
    'fmax': FMAX, 
    'cmap': 'viridis'
}
RECTANGLE_CONFIG = {
    'linewidth': 1., 
    'edgecolor': 'yellow', 
    'facecolor': 'yellow', 
    'alpha': 0.2
}

In [ ]:
class RowGenerator:
    def __init__(self, df: pd.DataFrame, data_dir: Path=DATA_DIR, sr: str=SR):
        self.df = df
        self.data_dir = data_dir
        self.sr = sr
        
    def __getitem__(self, idx) -> Tuple:
        row = self.df.loc[idx]
        cut_audio, start_t = self.read_cut_audio(row)
        
        # get audio waveplot
        wandb_audio = wandb.Audio(cut_audio, sample_rate=self.sr)
        # get melspectrogram with localized box
        offset_t = row.t_min - (start_t/self.sr)
        pt = (offset_t, row.f_min)
        width = row.t_max-row.t_min
        height = row.f_max-row.f_min
        melspec_fig = self.render_melspec_with_box_plot(cut_audio, pt,
                                                        width, height)
        melspec_wandb_image = self.fig_to_wandb_image(melspec_fig)
        # other metadata
        recording_id = row.recording_id
        species_id = row.species_id
        songtype_id = row.songtype_id
        label_type = row.label_type
        
        out = (recording_id, species_id, songtype_id,
               label_type, wandb_audio, melspec_wandb_image)
        return out
            
    def __len__(self):
        return self.df.shape[0]
    
    def read_cut_audio(self, row: pd.Series) -> Tuple[np.ndarray, int]:        
        fn = self.data_dir/ f'train/{row.recording_id}.flac'
        assert fn.is_file()
        audio, sr = librosa.load(fn, sr=self.sr)
        t_min, t_max = int(row.t_min*self.sr), int(row.t_max*self.sr)
        t_min = int(max(t_min-self.sr, 0))
        t_max = int(min(t_max+self.sr, 60*self.sr))
        return audio[t_min:t_max], t_min
    
    @staticmethod
    def fig_to_wandb_image(fig: plt.Figure) -> wandb.Image:
        with tempfile.NamedTemporaryFile(suffix='.png') as tmpfile:
            fig.savefig(tmpfile.name)
            img = Image.open(tmpfile.name)
            wandb_img = wandb.Image(img)
            # prevent figure being displayed in notebook
            plt.close()
        return wandb_img
    
    @staticmethod
    def render_melspec_with_box_plot(audio: np.ndarray, pt: Tuple[float, float], width: float, height: float) -> plt.Figure:
        """ 
        render melspectrogram to visible image 
        ref: https://www.kaggle.com/gpreda/explore-the-rainforest-soundscape
        """
        fig, ax = plt.subplots(figsize=(16, 9))
        spec = librosa.feature.melspectrogram(audio, **LIBROSA_CONFIG)
        dbs = librosa.amplitude_to_db(abs(spec))
        librosa.display.specshow(dbs, x_axis='time', y_axis='mel',
                                 **DISPLAY_CONFIG)
        rec = patches.Rectangle(pt, width=width, height=height,
                                **RECTANGLE_CONFIG)
        ax.add_patch(rec)
        plt.tight_layout()
        plt.colorbar()
        return fig
    
    @staticmethod
    def render_waveplot(audio: np.ndarray) -> plt.Figure:
        fig, ax = subplots(figsize=(16, 9))
        ax = librosa.display.waveplot(audio, sr=self.sr)
        return fig

In [ ]:
tp_df = pd.read_csv(TP_PATH)
tp_df['label_type'] = 'TP'
fp_df = pd.read_csv(FP_PATH)
fp_df['label_type'] = 'FP'
df = pd.concat([tp_df, fp_df]).reset_index(drop=True)
df = df.groupby(by=['species_id', 'songtype_id', 'label_type']).sample(SAMPLE_N_PER_GROUP).reset_index(drop=True)

In [ ]:
# read env var set in kernel
user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_key")
wandb.login(key=wandb_api)
run = wandb.init(project=WANDB_PROJECT, name=WANDB_RUN_NAME, job_type=WANDB_JOB_TYPE)

In [ ]:
#filter_df = df[df.label_type == 'TP'].reset_index(drop=True)
processor = RowGenerator(df)
columns = ['recording_id', 'species_id', 'songtype_id', 'label_type', 'audio', 'melspectrogram']
table = wandb.Table(columns)

sample_n = df.shape[0]
print(f'Creating wandb.Table of {sample_n} entries')
for idx in tqdm(range(sample_n)):
    row_tuple = processor[idx];
    table.add_data(*row_tuple)
    if idx % 500 == 0:
        # this line is so noisy
        plt.clf()
        print(f'Completed {idx+1} rows')
        
run.log({'EDA_Table': table})        
print('Run completed')